# Kognitive Systeme - Lab 1

Abgabetermin: Das fertig erstellte, dokumentierte und lauffähige Notebook schicken Sie bitte bis
Sonntag, 1.11.2020 (23:59:59 MEZ) an norbert.oswald@unibw.de

Das Ziel dieser Laborübung besteht darin, Informationen aus RDF Daten unterschiedlicher Quellen durch entsprechende SPARQL-Queries automatisch zu extrahieren. Initial ist die RDF-Datei `persons.n3`, die Personen, deren Alter, Interessen, Reiseziele sowie deren Adressen beschreibt, gegeben. Diese Datei soll zunächst in einen lokalen Triple-Store integriert werden. Als Triple-Store können Sie z.B. [GraphDB](https://graphdb.ontotext.com/) verwenden. Anschließend sollen Sie ein Python-Programm schreiben, dass die folgende Frage beantwortet: Welche der Städte, die in der RDF-Datei vorkommen, hat den bevölkerungsmäßig größten Vorort? Da nicht alle Informationen in der initialen RDF-Datei vorliegen, muss auf externe Informationsquellen zugegriffen werden. Als externe Datenquelle verwenden Sie für dieses Lab [WikiData](http://query.wikidata.org/sparql). Da stets verschiedene Namensräume verwendet werden, ist zu recherchieren, welche konkreten Properties für die gesuchten Informationen konkret benötigt werden. Falls Sie sich für WikiData entschieden haben gilt zu beachten: die Q-Werte für WikiData müssen für alle in der RDF-Datei beschriebenen Städte automatisch ermittelt werden! 

Falls Sie GraphDB verwenden wollen, laden Sie von der Webseite (https://graphdb.ontotext.com/) die freie Version herunter (diese erfordert Java 8) und installieren diese. Starten Sie dann den TripleStore durch Ausführen des Skripts `graphdb` im Verzeichnis `bin` der GraphDB Installation. Sie gelangen dann im Browser unter `localhost:7200` auf ihren lokalen GraphDB TripleStore. Erstellen und Aktivieren Sie zunächst ein Repository. Importieren Sie dann die Datei `persons.n3` mit den RDF-Daten in das erstellte Repository. Testen Sie das erfolgreiche Einladen der Datei, indem Sie in GraphDB die Option SPARQL auswählen, dort die nachfolgenden Zeilen hinein kopieren und und ausführen. 

Das Importieren war erfolgreich, wenn Sie als Ausgabe alle Personen aus der RDF-Datei erhalten. 

#### Aufgabe 1

Formulieren Sie eine SPARQL-Abfrage, mit der Sie den Wohnort jeder Person aus der RDF-Datei extrahieren können. Zum Testen der Query können Sie den GraphDB TripleStore verwenden. Erstellen Sie anschließend ein Python Programm, das automatisch die SPARQL-Abfrage ausführt und das Ergebnis der Abfrage ausgibt.

In [1]:
# Erstellen Sie hier Ihr Python Programm

from SPARQLWrapper import SPARQLWrapper, JSON

# optional display person and person name
# http://localhost:7200/sparql
query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX ex: <http://example.org/>
SELECT ?person ?name ?city WHERE { 
	?person a foaf:Person ;
        ex:address ?address .
    ?address ex:city ?city .
    OPTIONAL {?person foaf:name ?name} .
}
""".strip()

sparql = SPARQLWrapper("http://localhost:7200/repositories/persons")
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

result = sparql.queryAndConvert()

print(result)
print()

## parsing the converted output from json
results = result.get("results", {}).get("bindings", [])
# list cities values
results = map(lambda result: result.get("city", {}), results)
results = map(lambda result: result.get("value", ""), results)
# remove namespace
results = map(lambda result: result.rsplit("/", 1), results)
results = map(lambda result: result[-1], results)

cities = list(results)
print(cities)

{'head': {'vars': ['person', 'name', 'city']}, 'results': {'bindings': [{'city': {'type': 'uri', 'value': 'http://example.org/Madrid'}, 'person': {'type': 'uri', 'value': 'http://example.org/Emma'}, 'name': {'type': 'literal', 'value': 'Emma_Dominguez'}}, {'city': {'type': 'uri', 'value': 'http://example.org/Paris'}, 'person': {'type': 'uri', 'value': 'http://example.org/Pierre'}}, {'city': {'type': 'uri', 'value': 'http://example.org/Rome'}, 'person': {'type': 'uri', 'value': 'http://example.org/Carlo'}, 'name': {'type': 'literal', 'value': 'Carlo_Conte'}}, {'city': {'type': 'uri', 'value': 'http://example.org/Munich'}, 'person': {'type': 'uri', 'value': 'http://example.org/Max'}, 'name': {'type': 'literal', 'value': 'Max_Meier'}}]}}

['Madrid', 'Paris', 'Rome', 'Munich']


#### Aufgabe 2

Nun sollen für jede Stadt die für WikiData benötigten Q-Codes ermittelt werden. Schreiben Sie ein Python-Programm, dass den Q-Code aller aufgeführten Städte ermittelt und ausgibt.

In [2]:
# Erstellen Sie hier Ihr Python Programm

# TODO: copy "user-config.py" to pywikibot root directory
import pywikibot

city2q = dict()

site = pywikibot.Site("en", "wikipedia")

for city in cities:
    
    page = pywikibot.Page(site, city)
    item = pywikibot.ItemPage.fromPage(page)
    
    city2q[city] = item.title()
    
print(city2q)

{'Madrid': 'Q2807', 'Paris': 'Q90', 'Rome': 'Q220', 'Munich': 'Q1726'}


#### Aufgabe 3

Nun sollen für jede Stadt die Stadtteile, ihre Fläche und deren Einwohnerzahl ermittelt werden. Finden Sie geeignete Properties, mit denen Sie Zugriff auf diese Informationen bekommen. Formulieren und testen Sie eine dafür geeignete SPARQL-Abfrage. Erstellen Sie anschließend in nachfolgender Zelle ein Python Programm, dass für jede Stadt deren Stadtteile, Fläche und Einwohnerzahl ausgibt.  

In [3]:
# Erstellen Sie hier Ihr Python Programm

import requests

url = "https://query.wikidata.org/sparql"
formatstring = "{:56} {:20.3f} {:12,}"
print("{:65} {:15} {:10}".format("Name", "Fläche in km²", "Bevölkerung"))
print()
all_suburbs = list()
for city, q in city2q.items():
    print('\033[1m' + "Stadtteile von " + city + '\033[0m')
    # print("% Search for suburbs in {}: %".format(city))

    # https://query.wikidata.org/
    # search property numbers by hovering hyperlinks
    # https://www.wikidata.org/wiki/Q2807
    query = """
    SELECT * WHERE {{
        wd:{} wdt:P150 ?suburbs .  
        ?suburbs rdfs:label ?name .
        OPTIONAL{{
            ?suburbs wdt:P2046 ?area ;  
                     wdt:P1082 ?population .
        }}
        FILTER(lang(?name)="en")  
        SERVICE wikibase:label {{
            bd:serviceParam wikibase:language "en" . 
        }}
    }}
    """.format(q)

    response = requests.get(url, params={"format": "json", "query": query})   
    # if status code != 200
    response.raise_for_status()    
    # get response json
    response = response.json()
    
    # get response results - bindings
    results = response.get("results", {}).get("bindings", [])
    # filter suburbs, area, population
    results = list(map(lambda result: (city,
        result.get("suburbs", {}).get("value", ""),
        result.get("name", {}).get("value", ""),
        float(result.get("area", {}).get("value", "0")),
        int(result.get("population", {}).get("value", "0"))
    ), results))
    results.sort(key=lambda tup: tup[2])
    all_suburbs.append(results)
    
    
    for city, suburbs, name, area, population in results:
        print(formatstring.format("  " + name, area, population))
    print()    
 

Name                                                              Fläche in km²   Bevölkerung

Stadtteile von Madrid
  Arganzuela                                                            6.462      151,965
  Carabanchel                                                          14.048      243,998
  Castellana                                                            0.773       16,922
  Centro                                                                5.228      131,928
  Chamberí                                                              4.679      137,401
  Ciudad Lineal                                                        11.426      212,529
  Fuencarral-El Pardo                                                 237.838      238,756
  Hortaleza                                                            27.420      180,462
  Latina                                                               25.427      233,808
  Moncloa-Aravaca                                               

#### Aufgabe 4

Welcher Stadtteil hat die meisten Einwohner? Erstellen Sie ein Python Programm, dass den Namen des Stadtteils, seine Einwohnerzahl und die dazugehörige Stadt ausgibt.

In [4]:
# Erstellen Sie hier Ihr Python Programm
